In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin

class NaiveBayesClassifier(BaseEstimator, ClassifierMixin):  
    def __init__(self):
        None

    def fit(self, X, y):
        return self
    
    def transform(self, X, *_):
        return X

    def predict(self, X):
        return []
    
    def get_params(self, deep=True):
        return {}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            self.setattr(parameter, value)
        return self

In [2]:
from sklearn.pipeline import Pipeline, FeatureUnion
from NGramGenerator import *

X  = pd.read_csv("data/train_set_x.csv")
Y  = pd.read_csv("data/train_set_y.csv")
X  = pd.merge(X, Y, on = 'Id')
Xn = pd.read_csv("data/test_set_x.csv")

ngram = NGramGenerator(2, True, 0, False)
nbayes = NaiveBayesClassifier()


p = Pipeline(
    [( 'ngram', ngram )] #+
    #[( 'nbayes', nbayes )]
)

#p.set_params(ngram__n=2)
A = p.fit_transform(X[:1000])


In [3]:
A

,0,1,2,3,4
01,1,7,0,1,0
02,0,1,0,3,0
03,0,1,0,1,0
04,0,0,0,1,0
05,0,1,2,1,0
06,0,1,0,0,0
07,0,0,0,2,0
09,0,1,1,0,0
0a,0,0,1,1,0
0c,0,0,0,1,0
